In [ ]:
import pandas as pd
from pathlib import Path
import os
import sys
import pickle
import matplotlib.pyplot as plt
import networkx as nx

sys.path.append(str(Path().resolve().parent / "src"))

from ingest import load_cites_data
from mock import generate_mock_trafficking_data
from wtie.viz.map_routes import create_trade_routes_map
from wtie.graph.model import build_trade_graph, summarise_graph

**Load and Filter Data**

In [ ]:
df_raw = pd.read_csv("../data/cites/sample.csv", low_memory=False)
df_legal = load_cites_data(df_raw)

df_legal.head()

**Load Synthetic Mock Data**

In [ ]:
df_synth = generate_mock_trafficking_data()
df_synth.head()

**Combine for Unified View**

In [ ]:
df_all = pd.concat([df_legal, df_synth], ignore_index=True)
df_all["synthetic"].value_counts()

**Top Importers/Exporters**

In [ ]:
df_all["Exporter"].value_counts().head(10)

In [ ]:
df_all["Importer"].value_counts().head(10)

**Most Common Trade Routes**

In [ ]:
df_all.groupby(["Exporter", "Importer", "synthetic"]).size().sort_values(
    ascending=False
).head(10)

**Most Common Species & Terms**

In [ ]:
df_all["Taxon"].value_counts().head(10)

In [ ]:
df_all["Term"].value_counts()

**Map**

In [ ]:
os.makedirs("outputs", exist_ok=True)

# Point to the project root and the flat file path
storage_path = Path("../storage").resolve()
file_path = storage_path / "combined_trade_data"

with open(file_path, "rb") as f:
    df = pickle.load(f)

df.columns = df.columns.str.strip().str.lower()

output_path = Path("../outputs/trade_routes_map.html").resolve()

create_trade_routes_map(df, output_path=output_path)

In [ ]:
print(df.columns.tolist())

In [ ]:
exporters = df["exporter"].dropna().unique()
importers = df["importer"].dropna().unique()

all_countries = sorted(set(exporters) | set(importers))
print(f"🌍 Total countries in data: {len(all_countries)}")
print(all_countries)
print(df["synthetic"].value_counts())

In [ ]:
graph = build_trade_graph(df)
print(graph)

In [ ]:
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(graph, seed=42)  # force-directed layout
nx.draw_networkx_nodes(graph, pos, node_size=300)
nx.draw_networkx_edges(graph, pos, arrowstyle="->", arrowsize=10)
nx.draw_networkx_labels(graph, pos, font_size=8)

plt.title("Wildlife Trade Network")
plt.axis("off")
plt.show()

In [ ]:
summarise_graph(graph)